In [ ]:
import fcntl, os
from datetime import datetime, timezone
from pathlib import Path

def _now():
    return datetime.now(tz=timezone.utc).isoformat()

#workdir=Path(os.path.join(str(Path.home()),"WORK/locktest"))
workdir=Path("/enterprise-fs/project/bench/locktest")
workdir.mkdir(parents=True,exist_ok=True)
testfile=Path(os.path.join(workdir,"testfile"))
initialtext=f"Hello world: {_now()}\n"
bytes_written=testfile.write_text(initialtext)

# Test single-threaded locking

locknames={
    fcntl.LOCK_SH: "shared",
    fcntl.LOCK_EX: "exclusive",
    fcntl.LOCK_UN: "unlocked"
}
fname=str(testfile)
for mode in ("r","r+"):
    modename="read-only" if mode=="r" else "read-write"
    print(f"Opening {fname} with mode {modename}")
    fd=open(fname, mode)
    print(f"{fname} opened mode {modename}")

    for lockrange in ("all", "some"):
        if lockrange == "some":
            num = 3
            start = 3
        else:
            num = 0
            start = 0
        for mechanism in ("lockf", "flock"):
            if mechanism == "flock" and lockrange == "some":
                print(f"Cannot do byte-range locking with flock(); skipping {modename} tests")
                continue
            for locktype in (fcntl.LOCK_SH, fcntl.LOCK_EX):
                print(f"Attempting {mechanism}() type {locknames[locktype]} on {fname} with len {num}, start {start}")
                try:
                    if mechanism=="lockf":
                        fcntl.lockf(fd, locktype, num, start, 0)
                    else:
                        fcntl.flock(fd, locktype)
                except OSError as e:
                    if e.errno==9 and locktype==fcntl.LOCK_EX and mode=="r" and mechanism=="lockf":
                        print("Received expected failure to get exclusive lockf() on read-only file.")
                        continue
                print(f"Lock acquired, attempting file operations for mode {modename}")
                fd.seek(0,0)  # Rewind to beginning
                if mode == "r":
                    print("Reading file")
                    assert fd.read() == initialtext  # Reads come first to test against initialtext
                else:
                    newtext=f"Updated world: {_now()}"
                    print("Writing file")
                    _=fd.write(newtext)
                    fd.seek(0,0)
                    print("Reading file")
                    assert fd.read() == newtext            
                print("File operations succeeded: Releasing lock")
                if mechanism == "lockf":
                    fcntl.lockf(fd, fcntl.LOCK_UN)
                else:
                    fcntl.flock(fd, fcntl.LOCK_UN)